In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer




In [6]:
df=pd.read_csv(r'C:\Users\admin\Documents\swiggy\swiggy.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148541 entries, 0 to 148540
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            148541 non-null  int64 
 1   name          148455 non-null  object
 2   city          148541 non-null  object
 3   rating        148455 non-null  object
 4   rating_count  148455 non-null  object
 5   cost          148410 non-null  object
 6   cuisine       148442 non-null  object
 7   lic_no        148312 non-null  object
 8   link          148541 non-null  object
 9   address       148455 non-null  object
 10  menu          148541 non-null  object
dtypes: int64(1), object(10)
memory usage: 12.5+ MB


In [7]:
df['cuisine'].isna().any().sum()


np.int64(0)

In [6]:
df=df[df['cuisine'].notna()]

In [65]:
df.isna().sum()

id                0
name              0
city              0
rating            0
rating_count      0
cost             44
cuisine           0
lic_no          143
link              0
address           0
menu              0
dtype: int64

In [15]:
df.duplicated().sum()

np.int64(0)

In [3]:
df = df.drop(columns=['name', 'link', 'menu', 'lic_no','id','address'])


In [7]:
df=df.copy()
df['rating'] = df['rating'].replace('--', np.nan)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['rating'] = df['rating'].fillna(round(df['rating'].mean(), 1))



In [8]:
df['rating_count'] = df['rating_count'].replace('Too Few Ratings', np.nan)
df['rating_count'] = (
    df['rating_count']
    .astype(str)
    .str.extract('(\d+)')
    .astype(float)
)
df['rating_count'] = df['rating_count'].fillna(df['rating_count'].median())
df['rating_count'] = df['rating_count']+10




In [9]:
df['cost'] = (
    df['cost']
    .astype(str)
    .str.replace('₹', '', regex=False)
)
df['cost'] = pd.to_numeric(df['cost'], errors='coerce')
df['cost'] = df['cost'].fillna(df['cost'].median())


In [10]:
df.reset_index(drop=True, inplace=True)


In [12]:
df.to_csv("swiggy_CLEANED.csv", index=False)

print("swiggy_CLEANED.csv saved")

swiggy_CLEANED.csv saved


In [5]:
cleaned_df = pd.read_csv('swiggy_CLEANED.csv')
cleaned_df.reset_index(drop=True, inplace=True)  



In [27]:
preprocessed_df = cleaned_df.copy()


In [48]:
print(cleaned_df.columns)

Index(['city', 'rating', 'rating_count', 'cost', 'cuisine'], dtype='object')


In [11]:
df['city'] = df['city'].str.split(',')

mlb_city = MultiLabelBinarizer()
city_encoded = mlb_city.fit_transform(df['city'])

city_df = pd.DataFrame(
    city_encoded,
    columns=mlb_city.classes_,
    index=df.index
)
df = pd.concat([df.drop('city', axis=1), city_df], axis=1)





In [17]:
df['cuisine'].apply(type).value_counts()


cuisine
<class 'str'>    148442
Name: count, dtype: int64

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

df['cuisine_list'] = df['cuisine'].apply(
    lambda x: [i.strip() for i in x] if isinstance(x, list)
    else [i.strip() for i in str(x).split(',')] if pd.notna(x)
    else []
)

mlb = MultiLabelBinarizer()
cuisine_encoded = mlb.fit_transform(df['cuisine_list'])

cuisine_df = pd.DataFrame(cuisine_encoded, columns=mlb.classes_, index=df.index)

df = pd.concat([df.drop(['cuisine', 'cuisine_list'], axis=1), cuisine_df], axis=1)


In [21]:
df.to_csv('preprocessed_data.csv', index=False)


In [9]:
preprocessed_df=pd.read_csv('preprocessed_data.csv')
preprocessed_df.reset_index(drop=True, inplace=True)  # <<-- add this


In [54]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148442 entries, 0 to 148441
Columns: 976 entries, rating to indian
dtypes: float64(3), int64(973)
memory usage: 1.1 GB


In [55]:
print("Same rows count:", len(cleaned_df) == len(preprocessed_df))


Same rows count: True


In [56]:
import pickle

with open('preprocessed_data.pkl', 'wb') as f:
    pickle.dump(preprocessed_df, f)


In [58]:
import pickle

with open('preprocessed_data.pkl', 'rb') as f:
    preprocessed_df = pickle.load(f)
